In [2]:
import pandas as pd
from PIL import Image
import pytesseract

# Define the path to the tesseract.exe file
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

img_path = '../dummy_data/Bsp_Arztbrief.jpg' 

# Open the image file
img = Image.open(img_path)

# Use pytesseract to convert the image into text
text = pytesseract.image_to_string(img)

In [17]:
import openai
import json
import pandas as pd

openai.api_key = "YOUR_API"

# Definieren Sie die Kategorien
kategorien = [
    "Name des behandelnden Arztes",
    "Name des angesprochenen Arztes",
    "Name des Patienten",
    "Diagnose",
    "Therapie",
    "Behandlungsverlauf",
    "Ergebnis"
]

# Definieren Sie den Text des Arztberichts
arztbericht = text

# Bereiten Sie den Prompt vor
prompt = f"Bitte kategorisieren Sie den folgenden Arztbericht in die angegebenen Kategorien: {arztbericht}"

# Fügen Sie die Kategorien zum Prompt hinzu
for kategorie in kategorien:
    prompt += f"\n\n{kategorie}:"

# Senden Sie den Prompt an die ChatGPT API
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Extrahieren Sie die Antwort
antwort = response.choices[0].message['content']

# Erstellen Sie ein Wörterbuch aus der Antwort
antwort_dict = dict(item.split(":") for item in antwort.split("\n") if item)

# Überprüfen Sie die "erratenen" Namen und Inhalte
# Hier können Sie eine Funktion hinzufügen, um die Gültigkeit der erratenen Namen und Inhalte zu überprüfen

# Erstellen Sie eine Tabelle aus dem Wörterbuch
tabelle = pd.DataFrame(list(antwort_dict.items()), columns=['Kategorie', 'Inhalt'])

# Drucken Sie die Tabelle
print(tabelle)


                        Kategorie  \
0    Name des behandelnden Arztes   
1  Name des angesprochenen Arztes   
2              Name des Patienten   
3                        Diagnose   
4                        Therapie   
5              Behandlungsverlauf   
6                        Ergebnis   

                                              Inhalt  
0                                        Prof. O. ne  
1                                          Dr. me or  
2                              N/A (nicht angegeben)  
3   Hochgradig kombiniertes Aortenvitium mit Druc...  
4   Implantation einer St. Jude Medical Gr. 25 in...  
5   Erfolgreicher herzchirurgischer Eingriff am 2...  
6   Komplikationsloser Verlauf erwartet, weitere ...  


In [18]:
print(antwort)

Name des behandelnden Arztes: Prof. O. ne
Name des angesprochenen Arztes: Dr. me or
Name des Patienten: N/A (nicht angegeben)
Diagnose: Hochgradig kombiniertes Aortenvitium mit Druckgradient von 110 mmHG, Dekompensation. Supravalvulire Aortenstenose
Therapie: Implantation einer St. Jude Medical Gr. 25 in Aortenposition, Patcherweiterung der Aorta ascendens supravalvular
Behandlungsverlauf: Erfolgreicher herzchirurgischer Eingriff am 26.10.49, Patient befindet sich auf der Intensivstation, extubiert, Sinusrhythmus vorhanden, neuimplantierte Klappe arbeitet einwandfrei, keine Hinweise auf Dysfunktion, keine Substanzen notwendig
Ergebnis: Komplikationsloser Verlauf erwartet, weitere Berichte folgen bei Verlegung
